# PWM

> 有源蜂鸣器    

有源蜂鸣器只要接上额定电源就可以发出声音。  

> 无源蜂鸣器    

无源蜂鸣器利用电磁感应原理，为音圈接入交变电流后形成的电磁铁与永磁铁相吸或相斥而推动振膜发声。

![alt text](image-77.png)

>对无源蜂鸣器的控制：GPIO控制、PWM控制

>> GPIO控制：

![GPIO控制](image-78.png)

这种方式浪费CPU时间

>> PWM控制：

PWM(Pulse Width Modulation)即脉冲宽度调制，通过对脉冲的宽度进行调制，来获得所需要波形。

![alt text](image-79.png)

> PWM参数：周期、占空比

![alt text](image-80.png)

> Exynos4412下的PWM控制器

以无源蜂鸣器为例

![alt text](image-81.png)

![alt text](image-82.png)

以一个实例来解释4412产生脉冲信号的机制：

![alt text](image-83.png)

1. 需要初始化TCNTB寄存器为159(50+109),初始化TCMPB寄存器为109
2. 使能定时器
3. TCNTB中的159会被加载到递减计数器，然后控制TOUT引脚输出低电平
4. 如果TCNTB的值减到各TCMPB的值相同时，输出会从低变成高
5. TCNTB会继续减少，直到减少到0并发出中断请求
6. 递减计数器重新加载数值，开始下一循环

产生的脉冲信号除了和TCNTB寄存器中的值大小有关外，还和递减计数器递减的速度有关

4412下的PWM控制器本质上也是上一个定时器

PWM使用PCLK作为时钟源，但需要对这个时钟源进行分频

PLCK(100M)-> 一级分频 -> 二级分频 -> PWM

> PWM的逻辑框图

![alt text](image-84.png)

> PWM相关寄存器

![alt text](image-85.png)

TCNTB 用于设置周期
TCMPB 用于设置占空比
TCNTO 用于读取当前计数器减到了几 
TCFG0 用于设置一级分频和死区宽度(保护大功率电器)
TCFG1用于设置二级分频
TCON 用于PWM的控制

1. TCFG0

    ![alt text](image-86.png)

2. TCFG1
   
   ![alt text](image-87.png)

3. TCON
   
   ![alt text](image-88.png)
   ![alt text](image-89.png)

   反相的功能用于调整先输入低电平再输出高电平

4. TCNTB

    ![alt text](image-90.png)

5. TCMPB
   
    ![alt text](image-91.png)

6. TCNTO

    ![alt text](image-92.png)

设置pwm以产生500hz脉冲频率

In [ ]:
#include "exynos_4412.h"
 
void Delay(unsigned int Time)
{
	while(Time --);
}
 
int main()
{
	/*1.将GPD0_0引脚设置成PWM0的输出引脚*/
	GPD0.CON = GPD0.CON & (~(0xF)) | (0x2);
	/*2.设置PWM0的一级分频	一级分频倍数设置为100倍*/
	PWM.TCFG0 = PWM.TCFG0 & (~(0xFF)) | 99;
	/*2.设置PWM0的二级分频	二级分频倍数设置为1倍  递减计数器递减频率 = PLCK / (99 + 1) / 1 = 1M*/
	PWM.TCFG1 = PWM.TCFG1 & (~(0xF));
	/*4.设置PWM0为自动重装载，使其能够产生连续的脉冲信号，第三位置1*/
	PWM.TCON = PWM.TCON | (1 << 3);
	/*5.设置PWM0的频率为500HZ*/ // 1/500 / 1/1000000 = 2000
	PWM.TCNTB0 = 2000;
	/*6.设置PWM0的占空比为50%*/
	PWM.TCMPB0 = 1000;
	/*7.将TCNTB0中的值手动装载到递减计数器*/
	PWM.TCON = PWM.TCON | (1 << 1);
	/*8.关闭手动更新,使用自动重装载，手动的部分只有第一个周期*/
	PWM.TCON = PWM.TCON & (~(1 << 1));
	/*9.使能PWM0，递减计数器开始递减*/
	PWM.TCON = PWM.TCON | 1;
 
	while(1)
	{
		PWM.TCON = PWM.TCON | 1;
		Delay(1000000);
		PWM.TCON = PWM.TCON & (~(1));
		Delay(1000000);		
	}
	return 0;
}

编程实现通过PWM控制蜂鸣器产生"嘀嘀"的声音

注：PWM的频率1000HZ，占空比%60

In [ ]:
#include "exynos_4412.h"
 
void Delay(unsigned int Time)
{
	while(Time --);
}
 
int main()
{
	/*1.将GPD0_0引脚设置成PWM0的输出引脚*/
	GPD0.CON = GPD0.CON & (~(0xF)) | (0x2);
	/*2.设置PWM0的一级分频	一级分频倍数设置为100倍*/
	PWM.TCFG0 = PWM.TCFG0 & (~(0xFF)) | 99;
	/*2.设置PWM0的二级分频	二级分频倍数设置为1倍  递减计数器递减频率 = PLCK / (99 + 1) / 1 = 1M*/
	PWM.TCFG1 = PWM.TCFG1 & (~(0xF));
	/*4.设置PWM0为自动重装载，使其能够产生连续的脉冲信号，第三位置1*/
	PWM.TCON = PWM.TCON | (1 << 3);
	/*5.设置PWM0的频率为1000HZ*/
	PWM.TCNTB0 = 1000;
	/*6.设置PWM0的占空比为60%*/
	PWM.TCMPB0 = 600;
	/*7.将TCNTB0中的值手动装载到递减计数器*/
	PWM.TCON = PWM.TCON | (1 << 1);
	/*8.关闭第一次手动更新，变成自动更新*/
	PWM.TCON = PWM.TCON & (~(1 << 1));
	/*9.使能PWM0，递减计数器开始递减*/
	PWM.TCON = PWM.TCON | 1;
 
	while(1)
	{
		PWM.TCON = PWM.TCON | 1; //使能PWM0,蜂鸣器响
		Delay(1000000);
		PWM.TCON = PWM.TCON & (~(1)); //关闭PWM0，蜂鸣器停止响
		Delay(1000000);		
	}
	return 0;
}

# IIC

> IIC总线概述    

IIC总线是Philips公司在八十年代初推出的一种`串行`、`半双工`总线主要用于近距离、低速的芯片之间的通信；

IIC总线硬件结构简单，成本较低，因此在各个领域得到了广泛的应用。

IIC总线有两根双向的信号线：
一根数据线SDA用于收发数据 data  
一根时钟线SCL用于通信双方时钟的同步 control

![alt text](image-93.png)

 IIC总线是一种多主机总线，连接在IIC总线上的器件分为主机和从机。
 
 主机有权发起和结束一次通信，而从机只能被主机呼叫；
 
 当总线上有多个主机同时启用总线时，IIC也具备冲突检测和仲裁的功能来防止错误产生； 
 
 每个连接到IIC总线上的器件都有一个唯一的地址(7bit)，且每个器件都可以作为主机也可以作为从机(同一时刻只能有一个主机),总线上的器件增加和删除不影响其他器件正常工作；
 
 IIC总线在通信时总线上发送数据的器件为发送器，接收数据的器件为接收器；

 ![alt text](image-94.png)

 >IIC总线通信过程：

1.主机发送起始信号启用总线  

2.主机发送一个字节数据指明从机地址和后续字节的传送方向。  

3.被寻址的从机发送应答信号回应主机  

4.发送器发送一个字节数据  

5.接收器发送应答信号回应发送器  

… … （循环步骤4、5）  

n.通信完成后主机发送停止信号释放总线

> IIC总线的寻址方式

IIC总线寻址方式：

IIC总线上传送的数据是广义的，既包括地址，又包括真正的数据。  

主机在发送起始信号后必须先发送一个字节的数据，该数据的高7位为从机地址，最低位表示后续字节的传送方向，'0'表示主机发送数据，'1'表示主机接收数据；

总线上所有的从机接收到该字节数据后都将这7位地址与自己的地址进行比较，如果相同，则认为自己被主机寻址，然后再根据第8位将自己定为发送器或接收器。

![alt text](image-95.png)

> IIC 总线信号的实现

>> 起始信号和中止信号

当空闲时，SCL和SDA信都为高电平

SCL为高电平时，SDA由`高变低`表示起始信号。  

SCL为高电平时，SDA由`低变高`表示停止信号。  

起始信号和停止信号都是由主机发出，起始信号产生后总线处于占用状态停止信号产生后总线处于空闲状态。

![alt text](image-96.png)

>> 字节传送与应答：

 IIC总线通信时每个字节为8位长度，数据传送时，先传送最高位，后传送低位，发送器发送完一个字节数据后接收器必须发送1位应答位来回应发送器即一帧共有9位。（低电平表示接收到，高电平不变表示没接到）

![alt text](image-97.png)

>> 同步信号

IIC传输也要和串口通信一样解决两个问题：

1. 怎样让接收方知道有数据传输过来
2. 怎样区分发送方发来的01 和 0011 这两种相似的电平变化

IIC总线在进行数据传送时
1. 时钟线SCL为低电平期间发送器向数据线上发送一位数据，在此期间数据线上的信号允许发生变化
2. 时钟线SCL为高电平期间接收器从数据线上读取一位数据，在此期间数据线上的信号不允许发生变化，必须保持稳定。

![alt text](image-98.png)

(时钟线SCL作用是告诉SDA啥时候收发数据，SCL经过8个高低电平，9次高低变化，完成一个字节发送)

若使用IIC总线让从机给主机发送一个字节的数据0xA2，画出SCL和SDA上的时序图。注：从机地址为0x63

0xA2-1010 0010

0x63-0110 0011-取7位加0-11000110

![alt text](image-99.png)

> IIC典型时序

![alt text](image-100.png)

![alt text](image-101.png)

起始信号、从机地址、终止信号无论那种情况均由主机发送。

## Exynos 4412 下的IIC控制器

Inter-Integrated circuit
> IIC 控制逻辑框图

![alt text](image-102.png)

> IIC 控制器在不同工作模式下的流程

1. 主机作为发送模式的工作流程

![alt text](image-103.png)

2. 主机作为接收模式的工作流程

![alt text](image-104.png)

3. 从机作为发送模式的工作流程

![alt text](image-105.png)

4. 从机作为接收模式的工作流程

![alt text](image-106.png)

> 寄存器

1. I2CCON  用于控制IIC控制器
   
    | NAME | BIT | DESCRIPTION |
    | --- | --- | --------- |
    | Acknowledge generation | 7 | 用于打开接收位 |
    | Tx clock source selection | 6 | 用于选择时钟源,影响通讯的速度 |
    | Tx/Rx Interrupt | 5 | 用于打开/关闭中断信号，告知信号传输完成 |
    | Interrupt pending flag | 4 | 用于告知传输是否完成的状态位， 完成后需要手工清零 |

2. I2CSTAT 用于控制IIC控制器和读取IIC的状态
   
    | NAME | BIT | DESCRIPTION |
    | --- | --- | --------- |
    | Acknowledge generation | 7 | 用于打开接收位 |
    | Tx clock source selection | 6 | 用于选择时钟源,影响通讯的速度 |
    | Tx/Rx Interrupt | 5 | 用于打开/关闭中断信号，告知信号传输完成 |
    | Interrupt pending flag | 4 | 用于告知传输是否完成的状态位， 完成后需要手工清零 |

3. I2CDS  用于数据的发送和接收
4. I2CADD 用于设置IIC控制器的地址(作为主机一般不用)

## MPU6050

> MPU 概述
MPU6050是一个运动处理传感器，其内部集成了3轴加速度传感器和3轴陀螺仪（角速度传感器）,以及一个可扩展数字运动处理器。


![alt text](image-109.png)

> MPU6050的主要参数

可测量X、Y、Z轴三个方向的角速度

可编程设置角速度测量范围为±250、±500、±1000、±2000°/sec

可测量X、Y、Z轴三个方向的加速度

可编程设置加速度测量范围为±2g、±4g、±8g、±16g

可编程设置低功耗模式

可编程设置采样频率

 ... ...

> MPU6050通信接口

MPU6050可以使用IIC总线和其他器件进行数据交互，我们可以使用IIC总线向MPU6050中的控制寄存器写入数据来设置MPU6050的工作参数，也可以使用IIC总线从MPU6050中的数据寄存器读取数据来获取加速度、角速度等信息。

![alt text](image-110.png)

In [ ]:
/****************MPU6050内部常用寄存器地址****************/

#define    SMPLRT_DIV        0x19    //陀螺仪采样率,典型值:0x07(125Hz)
#define    CONFIG            0x1A    //低通滤波频率,典型值:0x06(5Hz)
#define    GYRO_CONFIG       0x1B    //陀螺仪自检及测量范围,典型值:0x18(不自检,2000°/s)
#define    ACCEL_CONFIG      0x1C    //加速计自检及测量范围及高通滤波频率,典型值:0x0(不自检,2G,5Hz)
#define    ACCEL_XOUT_H      0x3B
#define    ACCEL_XOUT_L      0x3C
#define    ACCEL_YOUT_H      0x3D
#define    ACCEL_YOUT_L      0x3E
#define    ACCEL_ZOUT_H      0x3F
#define    ACCEL_ZOUT_L      0x40
#define    TEMP_OUT_H        0x41
#define    TEMP_OUT_L        0x42
#define    GYRO_XOUT_H       0x43
#define    GYRO_XOUT_L       0x44
#define    GYRO_YOUT_H       0x45
#define    GYRO_YOUT_L       0x46
#define    GYRO_ZOUT_H       0x47
#define    GYRO_ZOUT_L       0x48
#define    PWR_MGMT_1        0x6B    //电源管理,典型值:0x00(正常启用)
#define    SlaveAddress      0x68    //MPU6050-I2C地址

它的IIC地址有两个一个0x68一个0x69，他的地址由ADO决定

![alt text](image-111.png)

地址是0x68，为了避免冲突所以可以改地址。

> 向MPU6050的一个寄存器写一个字节的数据：

![alt text](image-107.png)

AD adress MPU6050的地址 

W write 此位写0

RA 寄存器地址，要写入MPU6050的相关寄存器的地址

1.主机(Exynos4412)发送起始信号

2.主机发送从机地址(MPU6050的地址)及读写方向(写)

3.从机(MPU6050)发送应答信号

4.主机发送一个字节数据(要写的寄存器的地址)

5.从机发送应答信号

6.主机发送一个字节数据(要写到寄存器的数据)

7.从机发送应答信号

8.主机发送停止信号

> 从MPU6050的一个寄存器读一个字节的数据

![alt text](image-108.png)

如果中途需要修改读写状态，需要重新开始一次通讯

1.主机(Exynos4412)发送起始信号

2.主机发送从机地址(MPU6050的地址)及读写方向(写)

3.从机(MPU6050)发送应答信号

4.主机发送一个字节数据(要写的寄存器的地址)

5.从机发送应答信号

6.主机(Exynos4412)发送起始信号

7.主机发送从机地址(MPU6050的地址)及读写方向(读)

8.从机(MPU6050)发送应答信号

9.从机发送一个字节数据(要读的寄存器中的数据)

10.主机发送非应答信号(不再接收更多的数据)

11.主机发送停止信号



In [ ]:

#include "exynos_4412.h"

/****************MPU6050内部寄存器地址****************/

#define	SMPLRT_DIV		0x19	//陀螺仪采样率，典型值：0x07(125Hz)
#define	CONFIG			0x1A	//低通滤波频率，典型值：0x06(5Hz)
#define	GYRO_CONFIG		0x1B	//陀螺仪自检及测量范围，典型值：0x18(不自检，2000deg/s)
#define	ACCEL_CONFIG	0x1C	//加速计自检、测量范围及高通滤波频率，典型值：0x18(不自检，2G，5Hz)
#define	ACCEL_XOUT_H	0x3B
#define	ACCEL_XOUT_L	0x3C
#define	ACCEL_YOUT_H	0x3D
#define	ACCEL_YOUT_L	0x3E
#define	ACCEL_ZOUT_H	0x3F
#define	ACCEL_ZOUT_L	0x40
#define	TEMP_OUT_H		0x41
#define	TEMP_OUT_L		0x42
#define	GYRO_XOUT_H		0x43
#define	GYRO_XOUT_L		0x44
#define	GYRO_YOUT_H		0x45
#define	GYRO_YOUT_L		0x46
#define	GYRO_ZOUT_H		0x47
#define	GYRO_ZOUT_L		0x48
#define	PWR_MGMT_1		0x6B	//电源管理，典型值：0x00(正常启用)
#define	WHO_AM_I		0x75	//IIC地址寄存器(默认数值0x68，只读)
#define	SlaveAddress	0x68	//MPU6050-I2C地址

/************************延时函数************************/

void mydelay_ms(int time)
{
	int i,j;
	while(time--)
	{
		for(i=0;i<5;i++)
			for(j=0;j<514;j++);
	}
}

/**********************************************************************
 * 函数功能：I2C向从机特定地址写一个字节
 * 输入参数：
 * 		slave_addr： I2C从机地址
 * 			  addr： 芯片内部特定地址
 * 			  data：写入的数据
**********************************************************************/

void iic_write (unsigned char slave_addr, unsigned char addr, unsigned char data)
{
	/*对时钟源进行512倍预分频  打开IIC中断（每次完成一个字节的收发后中断标志位会自动置位）*/
	I2C5.I2CCON = I2C5.I2CCON | (1 << 6) | (1 << 5);	// [6] 1  [5] 1 
	/*设置IIC模式为主机发送模式  使能IIC发送和接收*/
	I2C5.I2CSTAT = 0xd0;  	//[7:6] 11 [4] 1
	/*将第一个字节的数据写入发送寄存器  即从机地址和读写位（MPU6050-I2C地址+写位）*/
	I2C5.I2CDS = slave_addr << 1;
	/*设置IIC模式为主机发送模式  发送起始信号启用总线  使能IIC发送和接收*/
	I2C5.I2CSTAT = 0xf0;

	/*等待从机接受完一个字节后产生应答信号（应答后中断挂起位自动置位）*/
	while(!(I2C5.I2CCON & (1<<4)));

	/*将要发送的第二个字节数据（即MPU6050内部寄存器的地址）写入发送寄存器*/
	I2C5.I2CDS = addr;
	/*清除中断挂起标志位  开始下一个字节的发送*/
	I2C5.I2CCON = I2C5.I2CCON & (~(1<<4));
	/*等待从机接受完一个字节后产生应答信号（应答后中断挂起位自动置位）*/
	while(!(I2C5.I2CCON & (1<<4)));

	/*将要发送的第三个字节数据（即要写入到MPU6050内部指定的寄存器中的数据）写入发送寄存器*/
	I2C5.I2CDS = data;
	/*清除中断挂起标志位  开始下一个字节的发送*/
	I2C5.I2CCON = I2C5.I2CCON & (~(1<<4));
	/*等待从机接受完一个字节后产生应答信号（应答后中断挂起位自动置位）*/
	while(!(I2C5.I2CCON & (1<<4)));

	/*发送停止信号  结束本次通信*/
	I2C5.I2CSTAT = 0xD0;
	/*清除中断挂起标志位*/
	I2C5.I2CCON = I2C5.I2CCON & (~(1<<4));
	/*延时*/
	mydelay_ms(10);
}

/**********************************************************************
 * 函数功能：I2C从特定地址读取1个字节的数据
 * 输入参数：         slave_addr： I2C从机地址
 * 			       addr： 芯片内部特定地址
 * 返回参数： unsigned char： 读取的数值
**********************************************************************/

unsigned char iic_read(unsigned char slave_addr, unsigned char addr)
{

	//主机的接收模式
	unsigned char data = 0;
	//使用第6个I2C主机
	/*对时钟源进行512倍预分频  打开IIC中断（每次完成一个字节的收发后中断标志位会自动置位）*/
	I2C5.I2CCON = I2C5.I2CCON | (1<<6) | (1<<5);

	/*设置IIC模式为主机发送模式  使能IIC发送和接收*/
	I2C5.I2CSTAT = 0xd0;
	/*将第一个字节的数据写入发送寄存器  即从机地址和读写位（MPU6050-I2C地址+写位0）*/
	I2C5.I2CDS = slave_addr<<1;
	/*设置IIC模式为主机发送模式  发送起始信号启用总线  使能IIC发送和接收*/
	I2C5.I2CSTAT = 0xf0;
	/*等待从机接受完一个字节后产生应答信号（应答后中断挂起位自动置位）*/
	while(!(I2C5.I2CCON & (1<<4)));

	/*将要发送的第二个字节数据（即要读取的MPU6050内部寄存器的地址）写入发送寄存器*/
	I2C5.I2CDS = addr;
	/*清除中断挂起标志位  开始下一个字节的发送*/
	I2C5.I2CCON = I2C5.I2CCON & (~(1<<4));
	/*等待从机接受完一个字节后产生应答信号（应答后中断挂起位自动置位）*/
	while(!(I2C5.I2CCON & (1<<4)));

	/*清除中断挂起标志位  重新开始一次通信  改变数据传送方向*/
	I2C5.I2CCON = I2C5.I2CCON & (~(1<<4));

	/*将第一个字节的数据写入发送寄存器  即从机地址和读写位（MPU6050-I2C地址+读位1）*/
	I2C5.I2CDS = slave_addr << 1 | 0x01;
	/*设置IIC为主机接收模式  发送起始信号  使能IIC收发*/
	I2C5.I2CSTAT = 0xb0;
	/*等待从机接收到数据后应答*/
	while(!(I2C5.I2CCON & (1<<4)));


	/*禁止主机应答信号（即开启非应答  因为只接收一个字节）  清除中断标志位*/
	I2C5.I2CCON = I2C5.I2CCON & (~(1<<7))&(~(1<<4));
	/*等待接收从机发来的数据*/
	while(!(I2C5.I2CCON & (1<<4)));
	/*将从机发来的数据读取*/
	data = I2C5.I2CDS;

	/*直接发起停止信号结束本次通信*/
	I2C5.I2CSTAT = 0x90;
	/*清除中断挂起标志位*/
	I2C5.I2CCON = I2C5.I2CCON & (~(1<<4));
	/*延时等待停止信号稳定*/
	mydelay_ms(10);

	return data;

}


/**********************************************************************
 * 函数功能：MPU6050初始化
**********************************************************************/

void MPU6050_Init ()
{
	// 往寄存器中写入值
	iic_write(SlaveAddress, PWR_MGMT_1, 0x00); 		//设置使用内部时钟8M
	iic_write(SlaveAddress, SMPLRT_DIV, 0x07);		//设置陀螺仪采样率
	iic_write(SlaveAddress, CONFIG, 0x06);			//设置数字低通滤波器
	iic_write(SlaveAddress, GYRO_CONFIG, 0x18);		//设置陀螺仪量程+-2000度/s
	iic_write(SlaveAddress, ACCEL_CONFIG, 0x0);		//设置加速度量程+-2g
}



/**********************************************************************
 * 函数功能：主函数
 **********************************************************************/

int main(void)
{

	unsigned char zvalue_h,zvalue_l;						//存储读取结果
	short int zvalue;

	/*设置GPB_2引脚和GPB_3引脚功能为I2C传输引脚*/
	GPB.CON = (GPB.CON & ~(0xF<<12)) | 0x3<<12;			 	//设置GPB_3引脚功能为I2C_5_SCL
	GPB.CON = (GPB.CON & ~(0xF<<8))  | 0x3<<8;				//设置GPB_2引脚功能为I2C_5_SDA

	uart_init(); 											//初始化串口
	MPU6050_Init();											//初始化MPU6050

	printf("\n********** I2C test!! ***********\n");
	while(1)
	{
		zvalue_h = iic_read(SlaveAddress, GYRO_ZOUT_H);		//获取MPU6050-Z轴角速度高字节
		zvalue_l = iic_read(SlaveAddress, GYRO_ZOUT_L);		//获取MPU6050-Z轴角速度低字节
		zvalue  =  (zvalue_h<<8)|zvalue_l;					//获取MPU6050-Z轴角速度

		printf(" GYRO--Z  :Hex: %d	\n", zvalue);			//打印MPU6050-Z轴角速度
		mydelay_ms(100);
	}
	return 0;
}



综合项目：

实时监测开发板的放置状态，
当监测到开发板水平放置时，每隔一分钟向终端上打印一次当前的时间以及开发板的状态
如：“2023-04-05 23:45:00 Status: Normal”
当监测到开发板发生倾斜时，每隔一秒钟向终端上打印一次当前的时间以及开发板的状态
如：“2023-04-05 23:45:00 Status: Warning”
同时让蜂鸣器产生“滴滴”的警报声，在警报状态下，若按下Key2按键，解除蜂鸣器的警报声

提示：
开发板水平静止放置时MPU6050的Z轴上的加速度应该等于重力加速度的值(9.8m/s2)，而其X轴和Y轴上的加速度应该等于0
当开发板发生倾斜时MPU6050的Z轴上的加速度的分量会减小，而其X轴和Y轴上的加速度分量会增大
我们可以以此来判断开发板是否发生倾斜